<a href="https://colab.research.google.com/github/MuajiiTsai/110-1-NTU-DBME5028/blob/main_/%E8%82%87%E5%85%83%E5%88%A5%E7%95%B6%E6%88%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn import metrics
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.notebook import tqdm
from google.colab import drive

model_path = "/content/drive/My Drive/Deep_Learning/Midterm/model1.pt"

##Connect Google Drive

In [2]:
drive.mount('/content/drive')
# train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
# os.chdir(train_path)
# training_data_file = os.listdir(train_path)
# print(training_data_file[999])
# type(training_data_file)
# os.chdir(test_path)
# testing_data_file = os.listdir(test_path)
# print(len(training_data_file))

Mounted at /content/drive


##Label preprocessing
跑一次就好

In [ ]:
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'
raw_label_file = '/content/drive/My Drive/Deep_Learning/Midterm/train.csv'


img_labels = pd.read_csv(raw_label_file)
img_dir = os.listdir(train_path)

img_with_labels = pd.DataFrame(columns=['id', 'label'])
img_without_labels = pd.DataFrame(columns=['id', 'label'])
# add the row
img_labels.set_index('id', inplace=True)
for i in range(len(img_dir)):
    j = img_labels.loc[img_dir[i][:-11]][0]
    new = pd.DataFrame({
            'id': [img_dir[i]],
            'label': [j]
        })
    if not np.isnan(j):
        img_with_labels = img_with_labels.append(new, ignore_index=True)
    else:
        img_without_labels = img_without_labels.append(new, ignore_index=True)
img_with_labels.to_csv('/content/drive/My Drive/Deep_Learning/Midterm/img_with_labels.csv')
img_labels.reset_index(inplace=True)

KeyboardInterrupt: ignored

##Dataset

In [4]:
from torchvision.io import read_image, ImageReadMode

Resize = transforms.Compose([
  transforms.Resize([512,512])
])

class BoneDataset(Dataset):
    def __init__(self, annotations_file, directory, transform=None, target_transform=None):
        self.dir = directory
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = os.listdir(directory)
        self.transform = transform
        self.target_transform = target_transform
        # self.item = self.classification()
    def __len__(self):
        return len(self.img_labels)
    def __getitem__(self, idx):
        # img_path = self.img_dir[idx]
        # image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        # 
        # label = getlabel(img_path)
        img_path = self.img_labels.iloc[idx, 1]
        # print(img_path)
        image = read_image(os.path.join(self.dir, img_path), mode=ImageReadMode.RGB)
        image = image.type(torch.FloatTensor)
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    # def classification(self):
    #     '''
    #     img list -> img dataframe(pandas)
    #     pair the img and the label
    #     del the img labeled with "nan"
    #     '''
    #     img_with_labels = pd.DataFrame(columns=['img_list', 'label'])
    #     # add the row
    #     self.img_labels.set_index('id', inplace=True)
    #     for i in range(len(self.img_dir)):
    #         j = self.img_labels.loc[self.img_dir[i][:-11]][0]
    #         if not np.isnan(j):
    #             new = pd.DataFrame({
    #                 'img_list': [self.img_dir[i]],
    #                 'label': [j]
    #             })
    #             img_with_labels = img_with_labels.append(new, ignore_index=True)

    #     self.img_labels.reset_index(inplace=True)
    #     return img_with_labels   # The dataframe contained all labeled img

In [13]:
label_file = '/content/drive/My Drive/Deep_Learning/Midterm/img_with_labels.csv'
train_path = '/content/drive/My Drive/Deep_Learning/Midterm/train/'
test_path = '/content/drive/My Drive/Deep_Learning/Midterm/test/'

#model
batch_size = 8
learning_rate = 1e-4
num_show = 3
transform = transforms.Compose([
  transforms.Grayscale(num_output_channels=3),
  # transforms.PILToTensor(),
  transforms.Resize([512,512]),
  # transforms.ToPILImage(mode='RGB')
])


# Split All_train_dataset to train&validation
train_dataset = BoneDataset(label_file, train_path, transform=transform)
# train_dataset_size = int(len(All_train_dataset)*0.8)
# validation_dataset_size = len(All_train_dataset)-train_dataset_size
# print(validation_dataset_size)
# train_dataset, validation_dataset = random_split(All_train_dataset, [train_dataset_size, validation_dataset_size])

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True
)
'''
validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=1,
    shuffle = False
)
'''
# np_validation_dataset_array = next(iter(validation_dataloader))[1].numpy()
# print(np_validation_dataset_array)


# test_dataloader = torch.utils.data.DataLoader(
#     test_dataset,
#     batch_size=1,
#     shuffle = False
# )

model = models.efficientnet_b0(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
# model = nn.Sequential(model, nn.Softmax(dim=2))
model = model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
# print(train_dataloader.dataset)

In [9]:
# train_features, train_labels = next(iter(validation_dataloader))
# print(f"feature batch size: {train_features.size()}")
# print(f"labels batch size: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# # print(img)
# # plt.imshow(img)
# # plt.show()
# print(f"Label: {label}")

NameError: ignored

##Load Model

In [8]:
load_model = torch.load(model_path)
# print(type(load_model))
model = models.efficientnet_b0(pretrained=True)
model.conv1 = nn.Conv2d(
    1,
    64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
model = nn.Sequential(model, nn.Linear(1000,1))
model.load_state_dict(load_model)
model = model.cuda()

FileNotFoundError: ignored

##Training Model

In [14]:
num_epochs = 8

#train the model
best_valid_loss = np.inf

for idx_epoch in range(num_epochs):
    print(f"Epoch{idx_epoch}")

    #training phase
    model.train()

    train_losses = []
    for image_batch, label_batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        image_batch = image_batch.cuda()
        label_batch = label_batch.cuda()
        output_batch = model(image_batch)
        # print(type(output_batch))
        # print(output_batch.shape)
        output_batch = torch.flatten(output_batch)
        loss = loss_fn(output_batch, label_batch)
        loss.backward()
        optimizer.step()
        
        loss = loss.detach().cpu().numpy()
        train_losses.append(loss)

    train_loss = np.mean(train_losses)
    print(f"Training loss: {train_loss}")

    # y = np.array([1,1,2,2])
    # y_pred = np.array([0.1,0.4,0.35,2])
    # fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=2)
    # print(metrics.auc(fpr, tpr))

    '''
    #validating phase
    model.eval()

    valid_losses = []
    valid_accuracies = []
    valid_label = []
    valid_output = []
    for image_batch, label_batch in tqdm(validation_dataloader):
        image_batch = image_batch.cuda()
        label_batch = label_batch.cuda()
        output_batch = model(image_batch)
        output_batch = torch.flatten(output_batch)
        # print(output_batch.shape)
        loss = loss_fn(output_batch, label_batch)
        
        loss = loss.detach().cpu().numpy()  
        valid_losses.append(loss)

        output_batch = output_batch.detach().cpu().numpy()
        label_batch = label_batch.detach().cpu().numpy()
        valid_label.append(label_batch)
        valid_output.append(output_batch)
    
    ##### AUC #####
    print(output_batch.shape)
    auc = AUC(valid_output, valid_label)
    # valid_accuracies.append(auc)
    
    valid_loss = np.mean(valid_losses)
    valid_accuracy = np.mean(valid_accuracies)
    print(f"Testing loss: {valid_loss}, Testing Accuracy: {valid_accuracy}")
    '''
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = test_loss
    torch.save(model.state_dict(), model_path)
    print("The model is saved")

Epoch0


  0%|          | 0/884 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
def AUC(y, y_pred):
# y, y_pred must be numpy array
  fpr, tpr, thresholds = metrics.roc_curve(y, y_pred, pos_label=1)
  result = metrics.auc(fpr, tpr)
  print(result)
  return result

##Prediction of Testing Dataset

In [ ]:
model.eval()
model.cpu()
test_data = os.listdir(test_path)
datalist = pd.DataFrame(test_data, columns=['patient_id'])
datalist.set_index('patient_id')
test_output = pd.DataFrame(columns= ['id', 'label'])
for image_list in tqdm(test_data):
    # print(image_list)
    image = read_image(os.path.join(test_path, image_list), mode=ImageReadMode.RGB)
    image = transform(image)
    image = image.reshape(1, 3, 512, 512)
    image = image.type(torch.FloatTensor)
    # image = image.cuda()
    label_pred = model(image)
    new = pd.DataFrame({
            'id': [image_list],
            'label': [label_pred]
        })
    
    test_output = test_output.append(new, ignore_index=True)

NameError: ignored

#Playground

##Import CSV

In [ ]:
df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv')
# def a():
#     df = pd.read_csv (r'/content/drive/My Drive/Deep_Learning/Midterm/train.csv')
#     return df
# x = a()
# sorted_df = df.sort_values(by="label",kind='mergesort')
# i = df.loc["FOREARM_00f72d1153"]
df.set_index('id', inplace=True)
j = df.loc['FOREARM_000b20cbe3'][0]
print(type(j))
print(type("str"))
# j.reset_index(inplace=True)
df.reset_index(inplace=True)
# print(f"j0: {j[0]}\n")
# print(j['id'][0])
print(j)
print(df)

<class 'numpy.float64'>
<class 'str'>
nan
                      id  label
0     FOREARM_000b20cbe3    NaN
1     FOREARM_000be2e9ab    1.0
2     FOREARM_00192d1b21    NaN
3     FOREARM_0055c4c62a    1.0
4     FOREARM_00f72d1153    0.0
...                  ...    ...
3995    WRIST_ff39824aa4    0.0
3996    WRIST_ff5128afcd    1.0
3997    WRIST_ff60656c01    0.0
3998    WRIST_ffaf60a955    0.0
3999    WRIST_fff3b05bc0    0.0

[4000 rows x 2 columns]


##Import Data

In [ ]:
train_dataset = torch.tensor([])
transform1 = transforms.Compose([
  transforms.PILToTensor(),
])
print(train_dataset)
for i in range(100):
  img = Image.open(f"{train_path}/{training_data_file[i]}")
  img_tensor = transform1(img)
  a = torch.cat((train_dataset, img_tensor),0)
  # print(a.shape)

tensor([])


KeyboardInterrupt: ignored

In [ ]:
import torch
try:
    # Get GPU name, check if it's K80
    GPU_name = torch.cuda.get_device_name()
    if GPU_name[-3:] == "K80":
        print("Get K80! :'( RESTART!")
        exit()  # Restart the session
    else:
        print("Your GPU is {}!".format(GPU_name))
        print("Great! Keep going~")
except RuntimeError as e:
    if e.args == ("No CUDA GPUs are available",):
        print("You are training with CPU! "
              "Please restart!")
        exit()  # Restart the session
    else:
        print("What's wrong here?")
        print("Error message: \n", e)

Get K80! :'( RESTART!
